# Catalog connectors


This notebook tests some connectors to different catalogs andshows the way to execute a request through their Python API. 
- PEPS  
- Copernicus
- PAVICS



In [ ]:
# Uncomment and run this cell if birdy is not installed
# !pip install birdhouse-birdy

In [ ]:
# all imports for this notebook
import json
import requests
import os
import xml.etree.ElementTree as ET
from birdy import WPSClient

In [ ]:
# read config file first
with open('config.json', 'r', encoding="utf-8") as f:
    config = json.load(f)
print(config)

# Connecteur API PEPS

Reference: https://peps.cnes.fr/rocket/plus/img/PEPS-IF-0-0170-ATOS_01_00_[2].pdf
 (pages 8-11)

Example:
"recherche de produits **S2ST** avec un identifiant **MGRS** égal à **31TCJ** et avec une **couverture nuageuse** comprise entre **10% et 70%**":

https://peps.cnes.fr/resto/api/collections/S2ST/search.json?tileid=31TCJ&cloudCover=[10,70]


The list of search criteria and available values for each of the criteria is available at the following addresses  :
- Opensearch descriptors for collection [Sentinel 1](https://peps.cnes.fr/resto/api/collections/S1/describe.xml)
- Opensearch descriptors for collection [Sentinel 2](https://peps.cnes.fr/resto/api/collections/S2/describe.xml)
- Opensearch descriptors for collection [Sentinel 2 tuilés](https://peps.cnes.fr/resto/api/collections/S2ST/describe.xml)
- Opensearch descriptors fora collection [Sentinel 3](https://peps.cnes.fr/resto/api/collections/S3/describe.xml)


In [ ]:
parameters = {
    'tileid' : "31TCJ",
    'cloudCover' : '[10,70]'
}
collection = "S2ST"
out_format = "json"

In [ ]:
# construct the search URL
def PEPS_search_url(parameters, collection, out_format):
    search_URL = config['PEPS_url']
    if collection in config['PEPS_collections'] :
      search_URL += collection +"/"
    if out_format in config['PEPS_formats']:
      search_URL += "search." + out_format
    # add any parameters
    for key in parameters:
      search_URL += "&" + key + "=" + str(parameters[key])
    # replace first & with ?
    search_URL = search_URL.replace("&", "?", 1)
    return search_URL

In [ ]:
search_PEPS = PEPS_search_url(parameters, collection, out_format)
print(search_PEPS)

Save the query result as a json dictionary

In [ ]:
req = requests.get(search_PEPS, allow_redirects=True)
out_peps = req.json()

print("Keys of the json file: ")
for key in out_peps.keys():
    print("\t"+key)
print("Total nr of results: ", out_peps['properties']['totalResults'])
print("Nr of results on this page: ",len(out_peps['features']))
print("First result: ", out_peps['features'][0])

In [ ]:
# get next page's results
def get_next(result):
    for links in result['properties']['links']:
        if links['rel'] == 'self':
            this_page = links['href']
        if links['rel'] == 'next':
            next_page = links['href']
        if links['rel'] == 'last':
            last_page = links['href']
    print("Next: ", next_page)
    if this_page != last_page:
        return next_page
    else :
        print("Last: ", last_page)
        return None
        
get_next(out_peps)

In [ ]:
def get_all_peps_results(url):
    all_results = []
    req = requests.get(url, allow_redirects=True)
    out_peps = req.json()
    for feat in out_peps['features']:
        print(feat['properties']['productIdentifier'])
        all_results += feat
    req_peps = get_next(out_peps)
    while(req_peps):
        req = requests.get(req_peps, allow_redirects=True)
        out_peps = req.json()
        for feat in out_peps['features']:
            print(feat['properties']['productIdentifier'])
            all_results += feat
        req_peps = get_next(out_peps)
    return all_results
        
print(search_PEPS)      
peps_results = get_all_peps_results(search_PEPS)
print("Total results: ", len(peps_results))

# Connecteur COPERNICUS
Description: https://scihub.copernicus.eu/userguide/OpenSearchAPI

In [ ]:
q = "footprint:\"Intersects(41.9000, 12.5000)\""
search_COP = config['COP_url'] + q
search_COP

In [ ]:
r = requests.get(search_COP, allow_redirects=True, 
                 auth=("samuel.foucher", "2Wsx3edc"))
outfile = 'out_cop.xml'
if r.status_code == 200 and r.content:
    open(outfile, 'wb').write(r.content)
    print(outfile + " written.")
else:
    print("Search request to Copernicus platform unsuccessful. Check credentials!")

In [ ]:
#make sure out_cop.xml is there
if os.path.exists(outfile):
    # parse contents
    tree = ET.parse(outfile)
    root = tree.getroot()
    for child in root:
        print(child.tag, child.attrib, child.text)
        for gchild in child:
              print("\t", gchild.tag, gchild.attrib, gchild.text)

# Connecteur PAVICS


In [ ]:
wps = WPSClient(config['PAVICS_url'])
help(wps.pavicsearch)

In [ ]:
params = "variable:tasmin,project:CMIP5,experiment:rcp85,frequency:day,institute:CCCma,model:CanESM2"

In [ ]:
resp = wps.pavicsearch(constraints=params, limit=10, type="File")
[result, files] = resp.get(asobj=True)
files